In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import utils
from nets.NetMLPLowRank import NetMLPLowRank

In [4]:
batch_size = 128
batch_size_test = 1000

train_loader, test_loader = utils.build_dataset(dataset='MNIST')

In [5]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
lr = 0.01
n_epochs = 100

model = NetMLPLowRank(K=[8], d=[4])
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.361188
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.422387
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.176567

Test set: Average loss: -6.3668, Accuracy: 9245/10000 (92%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.196499
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.152591
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.160524

Test set: Average loss: -7.4443, Accuracy: 9392/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.186170
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.201154
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.247151

Test set: Average loss: -7.8705, Accuracy: 9432/10000 (94%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.100479
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.317804
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.111834

Test set: Average loss: -8.1655, Accuracy: 9529/10000 (95%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.072163
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.107400
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.110389

Test set: Average loss: -8

KeyboardInterrupt: 